In [1]:
%load_ext autoreload
%autoreload 2

In [9]:
from snyk_ai.advisories import Advisories
from snyk_ai.router import Router, RouteType
from snyk_ai import Models

advisories = Advisories("../data/advisories")
ROUTER = Router(model=None, advisories=advisories)

print(ROUTER._prompt_template)

You route user questions to the appropriate data source. Respond with a single JSON object.

DATA SOURCES:
1. Advisories (unstructured): vulnerability explanations, attack patterns, remediation

- **Cross-Site Scripting (XSS) in express-validator**
  A critical Cross-Site Scripting (XSS) vulnerability has been discovered in the `express-validator` package affecting versions prior to 4.5.0. This vulnerability allows attackers to inject malicious JavaScript code through validation error messages, potentially compromising user sessions and sensitive data.
- **SQL Injection in webapp-auth**
  A critical SQL injection vulnerability has been identified in the `webapp-auth` package affecting versions 2.0.0 through 2.3.0. This vulnerability allows attackers to execute arbitrary SQL commands through the authentication query builder, potentially leading to complete database compromise, data exfiltration, and authentication bypass.
- **Dependency Confusion in secure-config**
  A dependency confus

### Prompt template

---

You route user questions to the appropriate data source. Respond with a single JSON object.

#### DATA SOURCES:

##### 1. Advisories (unstructured): vulnerability explanations, attack patterns, remediation

- **Cross-Site Scripting (XSS) in express-validator**
  A critical Cross-Site Scripting (XSS) vulnerability has been discovered in the `express-validator` package affecting versions prior to 4.5.0. This vulnerability allows attackers to inject malicious JavaScript code through validation error messages, potentially compromising user sessions and sensitive data.
- **SQL Injection in webapp-auth**
  A critical SQL injection vulnerability has been identified in the `webapp-auth` package affecting versions 2.0.0 through 2.3.0. This vulnerability allows attackers to execute arbitrary SQL commands through the authentication query builder, potentially leading to complete database compromise, data exfiltration, and authentication bypass.
- **Dependency Confusion in secure-config**
  A dependency confusion vulnerability has been discovered in the `secure-config` package affecting versions 3.0.0 through 3.1.9. This vulnerability allows attackers to potentially inject malicious packages into the dependency resolution process by exploiting missing package integrity checks and scoped package naming conflicts.
- **Path Traversal in data-processor**
  A critical path traversal vulnerability has been identified in the `data-processor` package affecting versions 1.2.0 through 1.5.1. This vulnerability allows attackers to read, write, or delete files outside the intended directory by manipulating file paths with directory traversal sequences (e.g., `../`). This can lead to unauthorized access to sensitive files, data exfiltration, and potential system compromise.
- **Remote Code Execution in file-handler**
  A critical Remote Code Execution (RCE) vulnerability has been discovered in the `file-handler` package affecting versions 0.5.0 through 0.8.2. This vulnerability stems from unsafe deserialization of user-controlled data, allowing attackers to execute arbitrary code on the server. This can lead to complete system compromise, data theft, and lateral movement within the network.
- **Cross-Site Request Forgery (CSRF) in api-client**
  A Cross-Site Request Forgery (CSRF) vulnerability has been identified in the `api-client` Python package affecting versions 1.0.0 through 1.3.9. This vulnerability allows attackers to perform unauthorized actions on behalf of authenticated users by tricking them into submitting malicious requests. The package fails to implement proper CSRF token validation in API endpoints, making it vulnerable to state-changing request forgery attacks.
- **Server-Side Request Forgery (SSRF) in http-server**
  A Server-Side Request Forgery (SSRF) vulnerability has been discovered in the `http-server` Python package affecting versions 0.9.0 through 0.9.7. This vulnerability allows attackers to make the server send HTTP requests to arbitrary internal or external URLs, potentially leading to information disclosure, internal network scanning, or access to internal services that should not be exposed.
- **Insecure Deserialization in json-parser**
  An insecure deserialization vulnerability has been identified in the `json-parser` Python package affecting versions 3.0.0 through 3.2.0. This vulnerability allows attackers to execute arbitrary code or perform object injection attacks by providing malicious serialized data. The package uses unsafe deserialization methods that can instantiate arbitrary objects and execute code during the deserialization process.

##### 2. Database (structured): CVE records, packages, statistics

Schema:

- **vulnerabilities**: cve_id, package_id, vulnerability_type_id, severity_id, cvss_score, affected_versions, fixed_version, description, published_date
- **packages**: package_id, name, ecosystem
- **severity_levels**: severity_id, severity_name, min_cvss, max_cvss
- **vulnerability_types**: type_id, type_name, description

Available operations:

- Look up specific CVE details
- Search/filter vulnerabilities by ecosystem, severity, type, CVSS range
- List packages by ecosystem
- Get statistics grouped by ecosystem, severity, or type

#### FIELD REQUIREMENTS BY ROUTE TYPE:

| route_type   | unstructured_query | structured_query |
|--------------|--------------------| -----------------|
| unstructured | required string    | must be null     |
| structured   | must be null       | required string  |
| hybrid       | required string    | required string  |
| none         | must be null       | must be null     |

WARNING: Your response will be validated. If validation fails, the request fails.

#### OUTPUT:

a single JSON object with these fields

- route_type: one of "unstructured", "structured", "hybrid", "none"
- unstructured_query: plain English question for advisories, or null
- structured_query: plain English description of data needed, or null
- reasoning: brief explanation (REQUIRED, never empty)

#### RULES:

- Query fields must be PLAIN ENGLISH sentences (never SQL, never JSON, never code)
- When in doubt, prefer "hybrid" - combining data with context gives more complete answers
- Only use "structured" or "unstructured" alone when clearly one-dimensional

#### EXAMPLES:

For "unstructured" (how attacks work, remediation, best practices):
`{"route_type": "unstructured", "unstructured_query": "How does SQL injection work?", "structured_query": null, "reasoning": "Asking about attack concepts"}`

For "structured" (data lookups, counts, filtering, specific CVEs):
`{"route_type": "structured", "unstructured_query": null, "structured_query": "List all critical severity vulnerabilities in npm packages", "reasoning": "Needs database filtering"}`

For "hybrid" - BOTH query fields REQUIRED (check yourself before responding):

- unstructured_query: non-empty string? YES
- structured_query: non-empty string? YES
`{"route_type": "hybrid", "unstructured_query": "How to remediate XSS vulnerabilities?", "structured_query": "Get details for CVE-2024-1234", "reasoning": "Needs both CVE data and remediation advice"}`

For "none" (off-topic, not about security):
`{"route_type": "none", "unstructured_query": null, "structured_query": null, "reasoning": "Not about security"}`

#### USER QUESTION:

_**{query}**_

#### BEFORE RESPONDING: 

If route_type="hybrid", verify BOTH query fields are non-empty strings

#### JSON:


---

## Example Queries to Support

Your system should handle various query types. Here are some examples to guide your testing:

- **Hybrid**: "How do I fix CVE-2024-1234 and what version should I upgrade to?"
- **Hybrid**: "What are all critical severity vulnerabilities in npm packages, and explain how the SQL injection one works?"
- **Structured-only**: "What is the average CVSS score for High severity vulnerabilities in pip packages?"
- **Unstructured-only**: "Explain how path traversal attacks work and show me a vulnerable code example."
- **Hybrid**: "Find all RCE vulnerabilities and explain the attack vector for the one with the highest CVSS score."


In [3]:
QUERIES = [
    "How do I fix CVE-2024-1234 and what version should I upgrade to?",
    "What are all critical severity vulnerabilities in npm packages, and explain how the SQL injection one works?",
    "What is the average CVSS score for High severity vulnerabilities in pip packages?",
    "Explain how path traversal attacks work and show me a vulnerable code example.",
    "Find all RCE vulnerabilities and explain the attack vector for the one with the highest CVSS score."
]

In [ ]:
models = [
    Models.Llama_3_2, 
    # Models.GPT_5_2, 
    # Models.Claude_Opus
]

for query in QUERIES:
    print(f"### {query}")
    print()

    for model in models:
        result = ROUTER.route(query, model)
        routing = result.route_type

        print(f"#### {model.name}: *{routing.value.upper()}*")

        print(f" - **Reasoning:** {result.reasoning}")
        if routing in [RouteType.STRUCTURED, RouteType.UNSTRUCTURED]:
            transformed = result.structured_query if routing is RouteType.STRUCTURED else result.unstructured_query
            print(f" - **Transformed:** {transformed}")
        if result.route_type is RouteType.HYBRID:
            print(f" - **Transformed**")
            print(f"   - **for structured:** {result.structured_query}")
            print(f"   - **for unstructured:** {result.unstructured_query}")

        print()
    
    print("---")
    print()

### How do I fix CVE-2024-1234 and what version should I upgrade to?

#### ollama:llama3.2: *HYBRID*
 - **Reasoning:** Needs both CVE data and remediation advice
 - **Transformed**
   - **for structured:** Get details for CVE-2024-1234
   - **for unstructured:** How to remediate XSS vulnerabilities?

#### openai:gpt-5.2: *HYBRID*
 - **Reasoning:** The user needs both remediation guidance (advisory-style explanation of how to fix) and the exact fixed version information from the CVE record in the database.
 - **Transformed**
   - **for structured:** Look up CVE-2024-1234 and return the affected versions and the fixed version to upgrade to, plus package name, ecosystem, severity, CVSS score, and description.
   - **for unstructured:** How do I remediate the vulnerability described in CVE-2024-1234, including recommended mitigation steps and best practices?

#### anthropic:claude-opus-4-5: *HYBRID*
 - **Reasoning:** User needs both the specific CVE data (affected/fixed versions) from the 

### How do I fix CVE-2024-1234 and what version should I upgrade to?

#### ollama:llama3.2: *HYBRID*
 - **Reasoning:** Needs both CVE data and remediation advice
 - **Transformed**
   - **for structured:** Get details for CVE-2024-1234
   - **for unstructured:** How to remediate XSS vulnerabilities?

#### openai:gpt-5.2: *HYBRID*
 - **Reasoning:** The user needs both remediation guidance (advisory-style explanation of how to fix) and the exact fixed version information from the CVE record in the database.
 - **Transformed**
   - **for structured:** Look up CVE-2024-1234 and return the affected versions and the fixed version to upgrade to, plus package name, ecosystem, severity, CVSS score, and description.
   - **for unstructured:** How do I remediate the vulnerability described in CVE-2024-1234, including recommended mitigation steps and best practices?

#### anthropic:claude-opus-4-5: *HYBRID*
 - **Reasoning:** User needs both the specific CVE data (affected/fixed versions) from the database and remediation guidance from advisories
 - **Transformed**
   - **for structured:** Get details for CVE-2024-1234 including affected versions and fixed version
   - **for unstructured:** How do I fix and remediate the vulnerability in CVE-2024-1234?

---

### What are all critical severity vulnerabilities in npm packages, and explain how the SQL injection one works?

#### ollama:llama3.2: *HYBRID*
 - **Reasoning:** Needs both CVE data and remediation advice, specifically about SQL injection
 - **Transformed**
   - **for structured:** List all critical severity vulnerabilities in npm packages and provide details for CVE-2022-1234
   - **for unstructured:** How do SQL injection attacks work?

#### openai:gpt-5.2: *HYBRID*
 - **Reasoning:** The user needs a database-driven list of critical npm vulnerabilities (structured) and a conceptual explanation of how the SQL injection operates (unstructured).
 - **Transformed**
   - **for structured:** List all critical severity vulnerabilities affecting npm packages, including their CVE IDs if available, affected versions, fixed versions, and brief descriptions.
   - **for unstructured:** Explain how the SQL injection vulnerability works in the webapp-auth package and how it can be exploited and remediated.

#### anthropic:claude-opus-4-5: *HYBRID*
 - **Reasoning:** User wants both a filtered list of critical npm vulnerabilities (requires database query) and an explanation of how SQL injection attacks work (requires advisory content)
 - **Transformed**
   - **for structured:** List all critical severity vulnerabilities in npm packages
   - **for unstructured:** How does SQL injection work and what are the attack patterns for the SQL injection vulnerability in webapp-auth?

---

### What is the average CVSS score for High severity vulnerabilities in pip packages?

#### ollama:llama3.2: *STRUCTURED*
 - **Reasoning:** Needs database filtering
 - **Transformed:** Get statistics grouped by ecosystem and severity for High severity vulnerabilities in pip packages

#### openai:gpt-5.2: *STRUCTURED*
 - **Reasoning:** The user requests an aggregate statistic (average CVSS) filtered by severity level and ecosystem, which requires querying the structured vulnerabilities database.
 - **Transformed:** Compute the average CVSS score for vulnerabilities with High severity in the pip (Python) ecosystem.

#### anthropic:claude-opus-4-5: *STRUCTURED*
 - **Reasoning:** This is a statistical calculation requiring database aggregation - filtering by severity level (High) and ecosystem (pip), then computing average CVSS score. No conceptual explanation or remediation advice needed.
 - **Transformed:** Calculate the average CVSS score for vulnerabilities with High severity in pip ecosystem packages

---

### Explain how path traversal attacks work and show me a vulnerable code example.

#### ollama:llama3.2: *UNSTRUCTURED*
 - **Reasoning:** Explaining attack concepts
 - **Transformed:** How do path traversal attacks work?

#### openai:gpt-5.2: *UNSTRUCTURED*
 - **Reasoning:** The user is asking for an explanation of an attack pattern and a vulnerable code example, which is best answered using the unstructured advisories content rather than querying CVE records or statistics.
 - **Transformed:** Explain how path traversal attacks work and provide an example of vulnerable code, along with typical remediation steps.

#### anthropic:claude-opus-4-5: *UNSTRUCTURED*
 - **Reasoning:** User is asking about attack concepts and code patterns for path traversal - this is educational/conceptual content found in advisory explanations, not a database lookup
 - **Transformed:** Explain how path traversal attacks work and show a vulnerable code example

---

### Find all RCE vulnerabilities and explain the attack vector for the one with the highest CVSS score.

#### ollama:llama3.2: *HYBRID*
 - **Reasoning:** Needs both CVE data and attack concept explanations
 - **Transformed**
   - **for structured:** Get details for the RCE vulnerability with the highest CVSS score
   - **for unstructured:** How does RCE work?

#### openai:gpt-5.2: *HYBRID*
 - **Reasoning:** Requires the database to enumerate all RCE records and pick the highest CVSS, and advisories-style context to explain the attack vector for that top-scoring issue.
 - **Transformed**
   - **for structured:** Find all vulnerabilities with type Remote Code Execution and identify which one has the highest CVSS score, returning its CVE ID, affected package, ecosystem, CVSS score, affected versions, and fixed version.
   - **for unstructured:** Explain the attack vector for the Remote Code Execution vulnerability that has the highest CVSS score, describing how an attacker exploits it and what the vulnerable mechanism is.

#### anthropic:claude-opus-4-5: *HYBRID*
 - **Reasoning:** Needs database query to find all RCE vulnerabilities and identify the highest CVSS score, plus advisory content to explain the attack vector for that vulnerability
 - **Transformed**
   - **for structured:** Find all RCE (Remote Code Execution) vulnerabilities and return their CVE details, package names, and CVSS scores, ordered by CVSS score descending to identify the highest scored one
   - **for unstructured:** How does Remote Code Execution (RCE) work through unsafe deserialization in file-handler package? Explain the attack vector and how attackers can execute arbitrary code.

---

